In [1]:
!pip install openai

from openai import OpenAI


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
# OpenAI API 클라이언트 생성
API_KEY = os.getenv("API_KEY")

# OpenAI API 클라이언트 생성
client = OpenAI(api_key=API_KEY)  # <- 본인 API 키 입력

In [ ]:
import json
from datetime import datetime

class WeatherBot:
    def __init__(self):
        self.client = OpenAI(api_key=API_KEY)  # <- 본인 API 키 입력

    def get_weather(self, location, unit="celsius"):
        """실제 날씨 API 대신 더미 데이터 반환"""
        weather_data = {
            "서울": {"temp": 15, "condition": "맑음"},
            "부산": {"temp": 18, "condition": "흐림"},
            "제주": {"temp": 20, "condition": "비"}
        }

        if location in weather_data:
            data = weather_data[location]
            if unit == "fahrenheit":
                data["temp"] = data["temp"] * 9/5 + 32
            return json.dumps(data, ensure_ascii=False)
        return json.dumps({"error": "위치를 찾을 수 없습니다"}, ensure_ascii=False)

    def chat_with_functions(self, user_message):
        """함수 호출 기능을 사용한 대화"""

        # 사용 가능한 함수 정의
        tools = [
            {
                "type": "function",
                "function": {
                    "name": "get_weather",
                    "description": "특정 위치의 날씨 정보를 가져옵니다",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "도시 이름 (예: 서울, 부산)"
                            },
                            "unit": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "온도 단위"
                            }
                        },
                        "required": ["location"]
                    }
                }
            }
        ]

        # 첫 번째 API 호출
        response = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "당신은 날씨 정보를 제공하는 어시스턴트입니다."},
                {"role": "user", "content": user_message}
            ],
            tools=tools,
            tool_choice="auto"
        )

        message = response.choices[0].message

        # 함수 호출이 필요한 경우
        if message.tool_calls:
            tool_responses = []
            for tool_call in message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)

                # 실제 함수 실행
                if function_name == "get_weather":
                    function_response = self.get_weather(**function_args)
                    tool_responses.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": function_response
                    })

            # 함수 결과를 포함하여 다시 API 호출
            second_response = self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "당신은 날씨 정보를 제공하는 어시스턴트입니다."},
                    {"role": "user", "content": user_message},
                    message,
                    *tool_responses # Add all tool responses
                ]
            )
            return second_response.choices[0].message.content

        return message.content

# 테스트
bot = WeatherBot()
print("🌤️ 날씨 봇 응답:")
print(bot.chat_with_functions("서울과 부산의 날씨를 알려주세요"))

🌤️ 날씨 봇 응답:
현재 서울의 날씨는 15도이며, 흐림으로 보입니다. 부산은 18도이며, 맑은 날씨를 보이고 있습니다. 추가적인 날씨 정보가 필요하시면 말씀해 주세요!
